In [1]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np
import unicodedata
from spacy.util import minibatch, compounding
import ast as ast
import pickle

In [2]:
# Função para substituir acentos

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    
    return str(text)

# Abrindo e gerando Datasets

### Entidade logradouro

In [3]:
with open ('../Datasets/Dataset_V2.0.txt', 'rb') as fp:
    dataset = pickle.load(fp)

print(len(dataset))

128016


In [4]:
dataset[0]

('R.GUILHERME MOREIRA,315 SUBLOJA,LOJA E 2.ANDAR 69005-300 MANAUS CENTRO AM',
 {'entities': [(0, 23, 'END_LOG')]})

In [5]:
end_lista = []

for i in range(len(dataset)):
    str_tratada = dataset[i][0]
    str_tratada = str_tratada.lower() # Tudo em minúsculo
    str_tratada = strip_accents(str_tratada) # Substituição de acentos e caracteres diferentes
    
    end_lista.append(str_tratada)

In [6]:
end_lista[:3]

['r.guilherme moreira,315 subloja,loja e 2.andar 69005-300 manaus centro am',
 '69005-300 manaus am r.guilherme moreira,315 centro subloja,loja e 2.andar',
 'r.guilherme moreira,315 centro 69005-300 subloja,loja e 2.andar manaus am']

In [7]:
iob = []

for i in range(len(dataset)):
    tupla = (end_lista[i], dataset[i][1])
    
    iob.append(tupla)
    
FULL_DATA = iob

In [8]:
print(FULL_DATA[0])
print(FULL_DATA[1])
print(FULL_DATA[2])

('r.guilherme moreira,315 subloja,loja e 2.andar 69005-300 manaus centro am', {'entities': [(0, 23, 'END_LOG')]})
('69005-300 manaus am r.guilherme moreira,315 centro subloja,loja e 2.andar', {'entities': [(20, 43, 'END_LOG')]})
('r.guilherme moreira,315 centro 69005-300 subloja,loja e 2.andar manaus am', {'entities': [(0, 23, 'END_LOG')]})


In [9]:
# Criação da base de teste e treinamento

n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(FULL_DATA) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])

    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n


base_teste, base_treinamento = gerador_bases(FULL_DATA, test_n)

random.shuffle(base_treinamento)
random.shuffle(base_teste)

print("Treinamento: " + str(len(base_treinamento)), "\nTeste: " + str(len(base_teste)), "\nTotal: " + str(len(FULL_DATA)))

Treinamento: 115214 
Teste: 12802 
Total: 128016


In [10]:
for i in range(5):
    print(base_treinamento[i])

('av. guajajaras, 200 tirirical ma 65055-285 quadra 220 - lojas 01 e 02 sao luis', {'entities': [(0, 19, 'END_LOG')]})
('cambe centro avenida inglaterra  86181-000 pr', {'entities': [(13, 31, 'END_LOG')]})
('centro sp mogi das cruzes 08710-500 av.vol.fernando p.franco,432 sala exclusivo', {'entities': [(36, 64, 'END_LOG')]})
('79965-000 itaquirai ms av.treze de maio,393 centro', {'entities': [(24, 44, 'END_LOG')]})
('rua dos andradas 1143 90020-015 rs porto alegre centro', {'entities': [(0, 21, 'END_LOG')]})


# Criando o modelo

In [11]:
# Define our variables

model = None
output_dir=Path(".")
n_iter= 250 # número de épocas
batch_size = 64

In [12]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

Created blank 'pt' model
Added new NER


In [13]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [14]:
print("Batch size: ", batch_size)
print("Épocas: ", n_iter)
print()

# add labels
for _, annotations in base_treinamento:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento)
        losses = {}
        batches = minibatch(base_treinamento, size=batch_size)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     
        
        print(itn+1, '\tLosses', losses)        

Batch size:  64
Épocas:  250

1 	Losses {'ner': 42406.08663019806}
2 	Losses {'ner': 15049.701008918018}
3 	Losses {'ner': 11350.498958502678}
4 	Losses {'ner': 9064.326153172273}
5 	Losses {'ner': 7616.480735804446}
6 	Losses {'ner': 6346.333682917659}
7 	Losses {'ner': 5661.365211955623}
8 	Losses {'ner': 5224.147627824183}
9 	Losses {'ner': 4786.875430345193}
10 	Losses {'ner': 4257.502126250984}
11 	Losses {'ner': 4274.676365970277}
12 	Losses {'ner': 4122.700997887913}
13 	Losses {'ner': 3948.6326896470528}
14 	Losses {'ner': 3632.2018594776814}
15 	Losses {'ner': 3526.779333861872}
16 	Losses {'ner': 3383.506706048531}
17 	Losses {'ner': 3158.5789782703105}
18 	Losses {'ner': 3201.920417003539}
19 	Losses {'ner': 2990.0705192696396}
20 	Losses {'ner': 2834.5102037985994}
21 	Losses {'ner': 2756.1972640193785}
22 	Losses {'ner': 2706.5008327412083}
23 	Losses {'ner': 2496.563174540845}
24 	Losses {'ner': 2471.653247965368}
25 	Losses {'ner': 2648.209574222119}
26 	Losses {'ner': 2

213 	Losses {'ner': 563.0215611536013}
214 	Losses {'ner': 520.6180420960169}
215 	Losses {'ner': 659.9350522392068}
216 	Losses {'ner': 579.6235185133171}
217 	Losses {'ner': 564.095032943676}
218 	Losses {'ner': 562.1823636748845}
219 	Losses {'ner': 547.6217024772541}
220 	Losses {'ner': 511.5338387494853}
221 	Losses {'ner': 578.3926111448992}
222 	Losses {'ner': 596.0341605029375}
223 	Losses {'ner': 579.7318777297733}
224 	Losses {'ner': 557.1131084603004}
225 	Losses {'ner': 472.17445095570025}
226 	Losses {'ner': 541.1947102670923}
227 	Losses {'ner': 585.7414298735154}
228 	Losses {'ner': 608.1470068059665}
229 	Losses {'ner': 595.8964407480152}
230 	Losses {'ner': 519.5385208411714}
231 	Losses {'ner': 633.3852996582453}
232 	Losses {'ner': 549.4795733238786}
233 	Losses {'ner': 623.8059795845331}
234 	Losses {'ner': 535.1691910501901}
235 	Losses {'ner': 525.7145849011621}
236 	Losses {'ner': 584.1992667302761}
237 	Losses {'ner': 574.5143701610921}
238 	Losses {'ner': 570.5

## Salvando modelo

In [15]:
nlp.to_disk('../Saved_models/')

## Carregando modelo para teste
#### Executar até antes de `Criando o modelo` caso já tenha treinado anteriormente.

In [10]:
nlp = spacy.load('../Saved_models/')

## Teste

Base de Teste

In [11]:
# test the trained model
for text, _ in base_teste:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print(text)
    print()

Entities [('avenida treze de maio, 1116', 'END_LOG')]
ce  avenida treze de maio, 1116 60040-531 fatima fortaleza

Entities [('r.rio de janeiro,750', 'END_LOG')]
belo horizonte mg 30160-041 r.rio de janeiro,750 centro 4.andar

Entities [('rua coronel oliver,112', 'END_LOG')]
rj santo antonio de padua rua coronel oliver,112  28470-000 centro

Entities [('rua edson da mota coreia, 840', 'END_LOG')]
centro ce caucaia 61600-040  rua edson da mota coreia, 840

Entities [('pca oswaldo cruz', 'END_LOG')]
sp sao paulo  04004-070 paraiso pca oswaldo cruz

Entities [('rua julio paulo da silva', 'END_LOG')]
57600-300  rua julio paulo da silva palmeira dos indios centro al

Entities [('rua quinze de novembro', 'END_LOG')]
rua quinze de novembro sp 18270-310 centro tatui

Entities [('pca.quinze de novembro,321', 'END_LOG')]
88010-400 pca.quinze de novembro,321 florianopolis centro sc

Entities [('r.avelino olimpio fernandes,140', 'END_LOG')]
mg monte azul centro 39500-000 r.avelino olimpio fernandes

Entities [('rua barao do rio branco, 462', 'END_LOG')]
sp rua barao do rio branco, 462 serrana 14150-000  centro

Entities [('av.dos autonomistas, 6856/6846', 'END_LOG')]
quitauna sp av.dos autonomistas, 6856/6846  06194-060 osasco

Entities [('av.nacoes unidas,2901', 'END_LOG')]
rs av.nacoes unidas,2901  rio branco 93310-500 novo hamburgo

Entities [('av dep emilio carlos 1861', 'END_LOG')]
02721-100  av dep emilio carlos 1861 sp sao paulo limo

Entities [('rod. amaral peixoto, 2.948', 'END_LOG')]
iguaba grande rj rod. amaral peixoto, 2.948 28960-000 centro km 98

Entities [('rua alfredo winck', 'END_LOG')]
centro chapada rs 99530-000 rua alfredo winck

Entities [('avenida das naes unidas', 'END_LOG')]
sp sao paulo avenida das naes unidas vila gertrudes 17 andar, torre a, sala 1 04794-000

Entities [('av. fernando costa, 400', 'END_LOG')]
mg itanhandu 37464-000 av. fernando costa, 400 centro

Entities [('praca dos tres poderes - ed. principal', 'END_LOG')]
praca dos tres poderes - ed.

Entities [('av.das comunidades,70', 'END_LOG')]
88350-360 brusque centro av.das comunidades,70 sc

Entities [('av. nove de julho, 564', 'END_LOG')]
centro  sp av. nove de julho, 564 15200-000 jose bonifacio

Entities [('av.rondon pacheco', 'END_LOG')]
38408-343 vigilato pereira uberlandia mg  av.rondon pacheco

Entities [('av.dr.mendonca lima,388', 'END_LOG')]
76850-000 guajara-mirim  av.dr.mendonca lima,388 ro centro

Entities [('av. presidente antonio carlos, 375,', 'END_LOG')]
rio de janeiro av. presidente antonio carlos, 375, 360305 20020-010 centro rj

Entities [('av.almirante marques de leao,389', 'END_LOG')]
av.almirante marques de leao,389 40140-230 salvador ba  barra

Entities [('av. rio branco', 'END_LOG')]
caxias do sul  95095-000 rs centro av. rio branco

Entities [('av lobo junior, 1715,  lj a', 'END_LOG')]
penha rio de janeiro rj 21020-123  av lobo junior, 1715,  lj a

Entities [('r. v.r.branco, 360 - lj 221 a 224, 227 a', 'END_LOG')]
rj shopping bay market r. v.r.branco,

Entities [('avenida assis brasil, n 1940', 'END_LOG')]
parte porto alegre avenida assis brasil, n 1940 rs so joo 91010-001

Entities [('av so paulo 911', 'END_LOG')]
sorocaba  18013-002 sp alem ponte av so paulo 911

Entities [('avenida carlos gomes  300', 'END_LOG')]
boa vista 13 andar 90480-000 rs avenida carlos gomes  300 porto alegre

Entities [('rua xv de novembro, 300', 'END_LOG')]
centro ibiuna sp rua xv de novembro, 300 18150-000

Entities [('rua antonio de barros, 319/321 - terreo', 'END_LOG')]
rua antonio de barros, 319/321 - terreo sao paulo 30630-000 centro sp

Entities [('av feitoria, 34', 'END_LOG')]
av feitoria, 34 rs  93040-193 s jose sao leopoldo

Entities [('av getulio vargas 146', 'END_LOG')]
centro es 29650-000  av getulio vargas 146 santa teresa

Entities [('rua general vitorino 1231', 'END_LOG')]
rua general vitorino 1231 restinga seca 6 regimento de cavalaria blindada 97542-311 rs alegrete

Entities [('av.duque de caxias,492', 'END_LOG')]
rio de janeiro rj 21615-

Entities [('av. raja gabaglia,', 'END_LOG')]
estoril 30494-310 av. raja gabaglia, 3790 mg belo horizonte

Entities [('av. borges de medeiros, 294', 'END_LOG')]
rs av. borges de medeiros, 294 porto alegre centro  90020-020

Entities [('av. senador ruy carneiro, 241', 'END_LOG')]
joao pessoa av. senador ruy carneiro, 241 58032-101 pb tambau

Entities [('av.adolfo konder,2552', 'END_LOG')]
urubici av.adolfo konder,2552 88650-000 esquina terreo sc

Entities [('rua sao paulo, 305', 'END_LOG')]
15930-000 centro candido rodrigues sp  rua sao paulo, 305

Entities [('av. santo antonio, 1199', 'END_LOG')]
centro sp 18730-000 av. santo antonio, 1199  itai

Entities [('rua coronel quirino, 683', 'END_LOG')]
rua coronel quirino, 683 parte cambui 13025-001 campinas sp

Entities [('rua rui barbosa, 100', 'END_LOG')]
rua rui barbosa, 100 88801-120 sc centro criciuma parte

Entities [('rua do carmo', 'END_LOG')]
diamantina  mg rua do carmo 39100-000 centro

Entities [('rua oito', 'END_LOG')]
jales sp c

...

Entities [('rua xv de novembro, 60/72', 'END_LOG')]
centro sp rua xv de novembro, 60/72 06850-100 itapecerica da serra

Entities [('av. dos imigrantes', 'END_LOG')]
av. dos imigrantes  antonio prado rs  95250-000

Entities [('r bento goncalves', 'END_LOG')]
centro 98700-000 rs ijui r bento goncalves

Entities [('avenida jacob macanhan', 'END_LOG')]
avenida jacob macanhan  centro pinhais pr 83320-352

Entities [('av joao naves de avila', 'END_LOG')]
mg av joao naves de avila uberlandia  38408-100 saraiva

Entities [('estrada manoel de s, 101 - loja', 'END_LOG')]
rj  belford roxo parque martinho 26182-000 estrada manoel de s, 101 - loja

Entities [('praca jose teodoro,20', 'END_LOG')]
praca jose teodoro,20 centro 18940-000 sao pedro do turvo  sp

Entities [('shcs qd 514,bl.c,loja 55', 'END_LOG')]
a sul shcs qd 514,bl.c,loja 55 n.51-terreo-w3 sul,n.52 e 54 subsolo-w2 70380-550 df brasilia

Entities [('av. padre jose nasser, 56', 'END_LOG')]
fama mg 37138-000  av. padre jose nasser, 56 cen

Entities [('estrada raul veiga, 360', 'END_LOG')]
estrada raul veiga, 360 lote 36 raul veiga rj 24710-480 sao goncalo

Entities [('avenida zeca abreu s/n', 'END_LOG')]
68560-000 santana do araguaia quadra 13 lote 08 pa centro avenida zeca abreu s/n

Entities [('r.dr.joao pessoa,77', 'END_LOG')]
centro bodoco r.dr.joao pessoa,77 em frente a tradicao 56220-000 pe

Entities [('rua schilling', 'END_LOG')]
rua schilling vila leopoldina sao paulo  05302-001 sp

Entities [('av.ipiranga,318 - bl.a - 14a. - cj.1401', 'END_LOG')]
sao paulo  sp centro av.ipiranga,318 - bl.a - 14a. - cj.1401 01046-010

Entities [('rua esprito santo', 'END_LOG')]
rua esprito santo centro belo horizonte n 616 - 8 andar mg 30160-920

Entities [('pca.joao pinheiro,8', 'END_LOG')]
predio aimores centro pca.joao pinheiro,8 mg 35200-000

Entities [('rua jornalista geraldo rocha, 200', 'END_LOG')]
rio de janeiro jardim america 21240-080 parte rua jornalista geraldo rocha, 200 rj

Entities [('r.silva jardim,51', 'END_LOG')

Entities [('rua carvalho de souza, 241 e 247- loja e', 'END_LOG')]
sobreloja madureira rua carvalho de souza, 241 e 247- loja e rj rio de janeiro 21350-180

Entities [('vp-008, s/n', 'END_LOG')]
vp-008, s/n 68508-020 nova maraba fl csi32q2lt1 maraba pa

Entities [('praa adroaldo campos, 457', 'END_LOG')]
capela praa adroaldo campos, 457 49700-000 se centro

Entities [('r. governador valadares, n 165', 'END_LOG')]
centro capelinha mg 39680-000  r. governador valadares, n 165

Entities [('avenida presidente castelo branco, 55', 'END_LOG')]
centro avenida presidente castelo branco, 55 69480-000 am tapaua

Entities [('rua monsenhor silvestri murari, 07', 'END_LOG')]
rua monsenhor silvestri murari, 07 sp 18130-000 centro sao roque

Entities [('rua do oratorio, 3859', 'END_LOG')]
sp  sao paulo v.prudente rua do oratorio, 3859 03117-000

Entities [('avenida brasilia, qd 43, lt 02', 'END_LOG')]
go centro avenida brasilia, qd 43, lt 02 edeia 75940-000 salas 02 e 03

Entities [('av. alameda xavi

Entities [('av engenheiro roberto freire', 'END_LOG')]
59080-900 av engenheiro roberto freire rn natal shopping cidade jardim lj 22 capim macio

Entities [('pca.rui barbosa,32', 'END_LOG')]
mogi-mirim pca.rui barbosa,32 sp centro  13800-002

Entities [('r.tiburcio targino,163', 'END_LOG')]
r.tiburcio targino,163 aquiraz 61700-000  ce centro

Entities [('av.cardeal eugenio pacelli, 139', 'END_LOG')]
32210-190 contagem pav.superior av.cardeal eugenio pacelli, 139 cidade industrial mg

Entities [('av.gal.mendonca lima,451', 'END_LOG')]
88745-000 centro  sc av.gal.mendonca lima,451 capivari de baixo

Entities [('av. fernando guilhon, 4.950', 'END_LOG')]
av. fernando guilhon, 4.950 centro marituba 67200-000 pa

Entities [('rua pio xii, 60', 'END_LOG')]
rua pio xii, 60 46900-000 seabra ba centro

Entities [('av.caetano barrella,95', 'END_LOG')]
06763-460  jd.da gloria taboao da serra av.caetano barrella,95 sp

Entities [('r.joaquim chagas filho,110', 'END_LOG')]
centro  morada nova ce 62940-

Entities [('av.rui barbosa,361', 'END_LOG')]
centro patrocinio 38740-000 mg av.rui barbosa,361

Entities [('estrada dos tres rios,920', 'END_LOG')]
estrada dos tres rios,920 22745-005 freguesia (jacarepagua) loja a rj rio de janeiro

Entities [('rua tocantins, 367', 'END_LOG')]
st central 77600-970 paraiso do tocantins rua tocantins, 367 to

Entities [('pc samuel sabatini,', 'END_LOG')]
sao bernardo do campo terreo jardim do mar sp pc samuel sabatini, 09750-700

Entities [('rua 15 de novembro, 21', 'END_LOG')]
rua 15 de novembro, 21 95330-000 rs veranopolis  centro

Entities [('av.newton bello,1078', 'END_LOG')]
santa luzia av.newton bello,1078 65390-000 ma centro

Entities [('estrada de jacarepagu, 6.215', 'END_LOG')]
rio de janeiro estrada de jacarepagu, 6.215 loja a rj freguesia/jacarepagu 22753-033



Frase qualquer

In [12]:
phrase = "SHVP Rua 12 435 casa 22 lote 1"

doc = nlp(phrase)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print(phrase)

Entities [('Rua 12 435', 'END_LOG')]
SHVP Rua 12 435 casa 22 lote 1


In [48]:
# Com texto IPTU

ph ="""
        "PREFEITURA DE Certidão de Dados Cadastrais do Imóvel - IPTU 2017\n\nSÃO PAULO\n\nFAZENDA",
        "Cadastro do Imóvel: 067.061.0048-1",
        "| Local do Imóvel:",
        "| AV JACANA, 764- AP 104 E VG",
        "BLOCO 1 UP LIFE SAO PAULO CEP 02273-001\nImóvel localizado na 22 Subdivisão da Zona Urbana",
        "| Endereço para entrega da notificação:",
        "| AV JACANA, 764- AP 104 E VG",
        "| BLOCO1 UPLIFE SAO PAULO CEP 02273-001",
        "| Contribuinte(s):",
        "| AGUA DAS FLORES EMPREENDIMENTOS IMOB LTDA",
        "| Dados cadastrais do terreno:",
        "| Área incorporada (m?): 1.669 Testada (m): 0,00",
        "| Área não incorporada (m?): 0 Fração ideal: 0,0024",
        "| Área total (m?): 7.669",
        "| Dados cadastrais da construção:",
        "| Área construída (m?):\n\n88\n\nPadrão da construção:\n\n2-C",
        "Área ocupada pela construção (m?): 5.134 Uso: residência",
        "| Ano da construção corrigido: 2016",
        "| Valores de m? (R$):",
        "| - de terreno: 700,00",
        "| - da construção: 1.508,00",
        "| Valores para fins de cálculo do IPTU (R$):",
        "| - da área incorporada: 23.675,00",
        "- da área não incorporada: 0,00",
        "| - da construção: 131.377,00",
        "|| Base de cálculo do IPTU: 155.052,00",
        "Ressalvado o direito da Fazenda Pública do Município de São Paulo atualizar os dados constantes do Cadastro\nImobiliário Fiscal, apurados ou verificados a qualquer tempo, inclusive em relação ao exercício abrangido por\nesta certidão, a Secretaria Municipal da Fazenda CERTIFICA que os dados cadastrais acima foram utilizados no\nlançamento do Imposto Predial e Territorial Urbano do imóvel do exercício de 2017.",
        "Certidão expedida via Internet - Portaria SF nº 008/2004, de 28/01/2004.",
        "A autenticidade desta certidão poderá ser confirmada, até o dia 05/02/2018, em",
        "http://www .prefeitura.sp.gov.br/cidade/secretarias/financas/servicos/certidoes/",
        "| Data de Emissão: 07/11/2017",
        "| Número do Documento: 2.2017.001599486-1",
        "|| Solicitante: PAULO AFONSO DECICINO (CPF 892.407.408-30)"
"""

In [46]:
# Função para tratamento de texto de entrada

def regex(str_):
    ph_tratada = str_.replace("\"", "")
    ph_tratada = ph_tratada.replace("-", " ")
    ph_tratada = ph_tratada.replace("|", "")
    ph_tratada = ph_tratada.replace(",", "")
    ph_tratada = ph_tratada.replace(":", "")
    ph_tratada = ph_tratada.replace("\", "")
#     ph_tratada = ph_tratada.replace(".", "")
    ph_tratada = ph_tratada.replace("  ", "")
    ph_tratada = ph_tratada.lower()
#     ph_tratada = re.sub(r'[\n]+', " ", ph_tratada)
    ph_tratada = re.sub(r'^[ ]+', "", ph_tratada)

    ph_tratada = strip_accents(ph_tratada
                              )    
    return ph_tratada

In [49]:
for each in ph.split("\n"):
    ph_tratada = regex(each)    
    doc_ph = nlp(ph_tratada)
    
    print('Entities', [(ent.text, ent.label_) for ent in doc_ph.ents])
    print(ph_tratada)
    print()

Entities []


Entities []
prefeitura de certidao de dados cadastrais do imovel iptu 2017

Entities []


Entities []
sao paulo

Entities []


Entities []
fazenda

Entities []
cadastro do imovel 067.061.0048 1

Entities []
local do imovel

Entities [('av jacana 764ap', 'END_LOG')]
av jacana 764ap 104 e vg

Entities [('cep 02273 001', 'END_LOG')]
bloco 1 up life sao paulo cep 02273 001

Entities []
imovel localizado na 22 subdivisao da zona urbana

Entities []
endereco para entrega da notificacao

Entities [('av jacana 764ap', 'END_LOG')]
av jacana 764ap 104 e vg

Entities [('cep 02273 001', 'END_LOG')]
bloco1 uplife sao paulo cep 02273 001

Entities []
contribuinte(s)

Entities []
agua das flores empreendimentos imob ltda

Entities []
dados cadastrais do terreno

Entities []
area incorporada (m?) 1.669 testada (m) 000

Entities []
area nao incorporada (m?) 0 fracao ideal 00024

Entities []
area total (m?) 7.669

Entities []
dados cadastrais da construcao

Entities []
area construida (m?)

In [17]:
doc_ph = nlp(ph_tratada)
print('Entities', [(ent.text, ent.label_) for ent in doc_ph.ents])

Entities [('104 e vg bloco 1', 'END_LOG'), ('104 e vg  bloco1', 'END_LOG'), ('incorporada 2367500 da area nao incorporada 000  da construcao 13137700', 'END_LOG')]
